### Text Summerizer

Library

In [1]:
import spacy
from heapq import nlargest
from collections import Counter
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS


In [2]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [14]:
text='''
Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3] During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

The terms 'male' and 'female' typically do not apply in sexually undifferentiated species in which the individuals are isomorphic (look the same) and the gametes are isogamous (indistinguishable in size and shape), such as the green alga Ulva lactuca. Some kinds of functional differences between individuals, such as in fungi,[6] may be referred to as mating types.[7]

There are several sex-determination systems. Most mammalian species have the XY sex-determination system, where the male usually carries an X and a Y chromosome (XY), and the female usually carries two X chromosomes (XX). Other chromosomal sex-determination systems in animals include the ZW system in birds, and the XO system in insects. Various environmental systems include temperature-dependent sex determination in reptiles and crustaceans.[8]
'''

In [4]:
document=nlp(text)
keywords=[]
stopwords=list(STOP_WORDS)
pos_tag=['NOUN','VERB','ADJ','PROPN']
for token in document:
   if (token.text in stopwords or token.text in punctuation):
     continue
   if (token.pos_ in pos_tag):
     keywords.append(token.text)


In [5]:
freq_words=Counter(keywords)
freq_words.most_common(6)

[('male', 6),
 ('female', 5),
 ('sex', 5),
 ('determination', 4),
 ('sexual', 3),
 ('produce', 3)]

In [6]:
max_freq_words=Counter(keywords).most_common(1)[0][1]
max_freq_words

6

In [7]:
for word in freq_words.keys():
  freq_words[word]=freq_words[word]/max_freq_words
freq_words.most_common(6)

[('male', 1.0),
 ('female', 0.8333333333333334),
 ('sex', 0.8333333333333334),
 ('determination', 0.6666666666666666),
 ('sexual', 0.5),
 ('produce', 0.5)]

In [8]:
sent_str={}
for sent in document.sents:
  for word in sent:
    if word.text in freq_words.keys():
      if sent in sent_str.keys():
        sent_str[sent]+=freq_words[word.text]
      else:
        sent_str[sent]=freq_words[word.text]
print(sent_str)

{
Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3]: 3.1666666666666665, During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent.: 4.166666666666666, By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4]: 7.666666666666667, An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex.: 7.333333333333335, Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

: 1.6666666666666667, The terms 'male' and '

In [9]:
summerized=nlargest(3, sent_str, key=sent_str.get)
print(' '.join([m.text for m in summerized]))

By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Most mammalian species have the XY sex-determination system, where the male usually carries an X and a Y chromosome (XY), and the female usually carries two X chromosomes (XX).


In [10]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
input_text=text
input_ids=tokenizer.batch_encode_plus (
    [input_text],
    return_tensors='pt',
    max_length=1024,
    truncation=True

)
summary_ids=model.generate(
    input_ids['input_ids'],
    num_beams=4,
    max_length=100,
    early_stopping=True

)
summary=tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
print('Summary:', summary)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Summary: Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes. During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. Most mammalian species have the XY sex-determination system, where the male usually carries an X and a Y chromosome.



SUMY LIBRARY

---



In [11]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 14.1 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=00eb1e2dda379c330f80ca8a57fca94f88ea5055bdd6fa697d7763578104e4f8
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5c3d2f18cc8d791a1b810098a60a107dc1f6986e01fd750c50419e23fe847ae5
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [16]:
!pip install --upgrade nltk

In [12]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

In [20]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
import nltk
nltk.data.path.append('/path/to/punkt.pickle')

In [22]:
# Input text to be summarized
input_text = text

# Parse the input text
parser = PlaintextParser.from_string(input_text, Tokenizer("english"))

# Create an LSA summarizer
summarizer = LsaSummarizer()

# Generate the summary
summary = summarizer(parser.document, sentences_count=3)  # You can adjust the number of sentences in the summary

# Output the summary
print("Original Text:")
print(input_text)
print("\nSummary:")
for sentence in summary:
    print(sentence)

Original Text:

Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3] During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

The terms 'male' and 'female' typically do not apply in sexually undifferentiated species in which the individu

 BERT Extractive Summarization

---



In [23]:
!pip install bert-extractive-summarizer

In [24]:
from summarizer import Summarizer

In [25]:
# Input text to be summarized
input_text = text

In [26]:
# Create a BERT extractive summarizer
summarizer = Summarizer()

# Generate the summary
summary = summarizer(input_text, min_length=50, max_length=150)  # You can adjust the min_length and max_length parameters

# Output the summary
print("Original Text:")
print(input_text)
print("\nSummary:")
print(summary)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Original Text:

Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3] During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

The terms 'male' and 'female' typically do not apply in sexually undifferentiated species in which the individu

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


T5 Abstractive Summarization

---



In [27]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Input text to be summarized
input_text = text

# Tokenize and summarize the input text using T5
inputs = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode and output the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Original Text:")
print(input_text)
print("\nSummary:")
print(summary)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Original Text:

Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3] During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

The terms 'male' and 'female' typically do not apply in sexually undifferentiated species in which the individu

TextTeaser

---



In [28]:
import requests

# Input text to be summarized
input_text = text

# Make a POST request to the TextTeaser API
response = requests.post("http://www.textteaser.com/api", data={"text": input_text})

# Extract and output the summary from the API response
summary = response.text
print("Original Text:")
print(input_text)
print("\nSummary:")
print(summary)

Original Text:

Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3] During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

The terms 'male' and 'female' typically do not apply in sexually undifferentiated species in which the individu

Gensim

---



In [35]:
!pip install gensim==3.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.6.0-cp310-cp310-linux_x86_64.whl size=23778004 sha256=18a8f8a0064c0c4852598899190e8374151fc4dfce08a2dee7d4066abac59712
  Stored in directory: /root/.cache/pip/wheels/00/e8/47/96f55c3144a5ea3537f549f7a97607011f5004b9f13fa8dcc5
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2


In [36]:
from gensim.summarization import summarize

In [37]:
# Input text to be summarized
input_text = text

# Generate the summary using TextRank algorithm
summary = summarize(input_text, ratio=0.3)  # You can adjust the ratio parameter based on the summary length you desire

# Output the summary
print("Original Text:")
print(input_text)
print("\nSummary:")
print(summary)

Original Text:

Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3] During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

The terms 'male' and 'female' typically do not apply in sexually undifferentiated species in which the individu